In [69]:
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split
from pandas import get_dummies, DataFrame

In [32]:


f=open('../Case Data/case1Data.txt','r')
data=f.read()
data1=data.split('\n')
arr=[]
for i in range(101):
    arr.append(data1[i].split(', '))
    


In [33]:
header=arr[0]
arr=arr[1:]
number_cols=[x[0:96] for x in arr]
class_cols=[x[96:] for x in arr]
y=np.array([x[0] for x in number_cols],dtype='float64')
x=np.array([x[1:] for x in number_cols],dtype='float64')
x_class=np.array(class_cols)

# one hot encoding





In [34]:
# # nan stuff

# c1=x_class[:,0]
# c2=x_class[:,1]
# c3=x_class[:,2]
# c4=x_class[:,3]
# c5=x_class[:,4]

# # for i in [c1,c2,c3,c4,c5]: 
# #     unique, counts = np.unique(i, return_counts=True)
# #     print (np.asarray((unique, counts)).T)

# count=0
# for i in range(101):
#     print(x_class[i])
# print(count)

In [35]:
# TODO: Figure out what is wrong with accuracy

# KNN
x_class_no_nans = x_class.copy()
def get_x_and_y(i):
    nans=np.where(i=='NaN')
    x_tmp = np.delete(x.copy(),nans,axis=0)
    y_tmp = np.delete(i.copy(),nans)
    return (x_tmp,y_tmp)

ns = range(1,25) #[1,3,5,10,20,60]
times_to_repeat = 200 
accuracy = np.zeros((x_class.shape[1],len(ns)))

for n_index, n in enumerate(ns):
    for col_index, col in enumerate(x_class.T):
        err = 0
        for loop in range(times_to_repeat):
            x_tmp,y_tmp = get_x_and_y(col)
            x_tmp, y_tmp = resample(x_tmp, y_tmp,n_samples=100)
            x_train, x_test,y_train, y_test = train_test_split(x_tmp,y_tmp,test_size=0.25)
            knn = KNeighborsClassifier(n_neighbors=n).fit(x_train,y_train)
            err += knn.score(x_test,y_test)
        err /= times_to_repeat
        accuracy[col_index, n_index] = err

In [36]:
# y_pred = knn.predict(x[nans])

for col_index, col in enumerate(x_class.T):
    best_k = np.argmax(accuracy[col_index]) + 1
    x_tmp,y_tmp = get_x_and_y(col)
    nans=np.where(col=='NaN')
    y_pred = KNeighborsClassifier(n_neighbors=best_k).fit(x_tmp,y_tmp).predict(x[nans])
    x_class_no_nans[nans,col_index] = y_pred

In [71]:
x_classes_one_hot = get_dummies(DataFrame(x_class_no_nans)).to_numpy()

x_knn = np.concatenate((x,x_classes_one_hot),axis=1)

x_one_hot = np.concatenate((x,get_dummies(DataFrame(x_class)).to_numpy()), axis=1)

In [72]:

def rmse(y_true,y_pred):
    return np.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
   

def train_model(model,x,y=y):
    #Linear Regression
    x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=0)
    m = model.fit(x_train, y_train)
    y_pred = m.predict(x_test)
    return rmse(y_test,y_pred)


lin_reg_rmse = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x_knn)
lin_reg_rmse_no_classes = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x)
lin_reg_rmse_no_knn = train_model(LinearRegression(fit_intercept=True,normalize=True, copy_X=True, n_jobs=None), x_one_hot)

In [81]:
#Ridge Regression

lambdas = np.logspace(-4,3,30)
times_to_repeat = 200 
accuracy = np.zeros((len(lambdas)))

for loop in range(times_to_repeat):
    err = 0
    for n_index, n in enumerate(lambdas):
        x_train, x_test,y_train, y_test = train_test_split(x_one_hot,y,test_size=0.25)
        model = Ridge(n).fit(x_train,y_train)
        y_pred =  model.predict(x_test)
        accuracy[n_index] += (np.sum(y_test - y_pred)**2) / len(y_test)

accuracy /= times_to_repeat




#Lasso Regression 





#Elastic Regression



#Random Forest

array([5.60898031e+02, 5.22440716e+02, 4.89463995e+02, 4.09974531e+02,
       5.21432176e+02, 5.64007346e+02, 4.65959280e+02, 3.20365782e+02,
       3.84531883e+02, 3.47064018e+02, 2.40065019e+02, 2.34605377e+02,
       2.08171156e+02, 1.41037648e+02, 1.05662309e+02, 8.95497320e+01,
       9.45578158e+01, 7.36484967e+01, 5.75488337e+01, 5.96846383e+01,
       4.46638005e+01, 4.37581397e+01, 4.18134235e+01, 3.57752929e+01,
       4.40244586e+01, 3.12808642e+01, 4.01616907e+01, 3.69788934e+01,
       3.40434959e+01, 3.37510223e+01, 1.00000000e-04, 1.74332882e-04,
       3.03919538e-04, 5.29831691e-04, 9.23670857e-04, 1.61026203e-03,
       2.80721620e-03, 4.89390092e-03, 8.53167852e-03, 1.48735211e-02,
       2.59294380e-02, 4.52035366e-02, 7.88046282e-02, 1.37382380e-01,
       2.39502662e-01, 4.17531894e-01, 7.27895384e-01, 1.26896100e+00,
       2.21221629e+00, 3.85662042e+00, 6.72335754e+00, 1.17210230e+01,
       2.04335972e+01, 3.56224789e+01, 6.21016942e+01, 1.08263673e+02,
      

In [96]:
a = np.concatenate((accuracy.reshape((1,len(accuracy))),lambdas.reshape((1,len(accuracy)))),axis=0).T

In [ ]:

#KNN 





In [ ]:
#LDA


#Regularized 













